# Simplexn

In [1]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

┌ Info: Precompiling LinearAlgebraicRepresentation [95167b0c-186a-594b-a231-52369c6de044]
└ @ Base loading.jl:1423


LinearAlgebraicRepresentation

## simplexFacets(simplices::Cells)::Cells
Compute the `(d-1)`-skeleton (unoriented set of `facets`) of a simplicial `d`-complex.

In [2]:
function simplexFacets(simplices)
	out = Array{Int64,1}[]
	for simplex in simplices
		for v in simplex
			facet = setdiff(simplex,v)
			push!(out, facet)
		end
	end
	# remove duplicate facets
	return sort(collect(Set(out)))
end

simplexFacets (generic function with 1 method)

In [ ]:
@btime simplexFacets([[1, 2, 3, 5],[2, 3, 5, 6],[3, 5, 6, 7],[2, 3, 4, 6],[3, 4, 6, 7],[4, 6, 7, 8]])

## simplex(n::Int, fullmodel=false::Bool)::Union{Lar.LAR, Lar.LARmodel}
Return a `LAR` model of the *`n`-dimensional simplex* in *`n`-space*.

When `fullmodel==true` return a `LARmodel`, including the faces, from dimension `1` to `n`.

In [3]:
function simplex(n, fullmodel=false)
	eye(n) = LinearAlgebra.Matrix{Int}(I,n,n)
	V = [zeros(n,1) eye(n)]
	CV = [collect(1:n+1)]
	if fullmodel == false
		return V,CV
	else
		h = n
		cells = [CV]
		while h != 0
			push!(cells, simplexFacets(cells[end]))
			h -= 1
		end
		return V,reverse(cells)
	end
end

simplex (generic function with 2 methods)

In [9]:
@btime simplex(3, true)

  9.889 μs (184 allocations: 18.27 KiB)


([0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0], [[[1], [2], [3], [4]], [[1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4]], [[1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]], [[1, 2, 3, 4]]])

## extrudeSimplicial(model::LAR, pattern::Array)::LAR
Algorithm for multimensional extrusion of a simplicial complex.
Can be applied to 0-, 1-, 2-, ... simplicial models, to get a 1-, 2-, 3-, .... model.

In [5]:
function extrudeSimplicial(model::Lar.LAR, pattern)
    V = [model[1][:,k] for k=1:size(model[1],2)]
    FV = model[2]
    d, m = length(FV[1]), length(pattern)
    coords = collect(cumsum(append!([0], abs.(pattern))))
    offset, outcells, rangelimit, i = length(V), [], d*m, 0
    for cell in FV
        i += 1
        tube = [v+k*offset for k in range(0, length=m+1) for v in cell]
        cellTube = [tube[k:k+d] for k in range(1, length=rangelimit)]
        if i==1 outcells = reshape(cellTube, d, m)
        else outcells = vcat(outcells, reshape(cellTube, d, m)) end
    end
    cellGroups = []
    for i in 1:size(outcells, 2)
        if pattern[i]>0
            cellGroups = vcat(cellGroups, outcells[:, i])
        end
    end
    outVertices = [vcat(v, [z]) for z in coords for v in V]
    cellGroups = convert(Array{Array{Int, 1}, 1}, cellGroups)
    outModel = outVertices, cellGroups
    hcat(outVertices...), cellGroups
end
function extrudeSimplicial(model::Union{Any,Lar.Cells}, pattern)
    V,FV = model
    d, m = length(FV[1]), length(pattern)
    coords = collect(cumsum(append!([0], abs.(pattern))))
    offset, outcells, rangelimit, i = length(V), [], d*m, 0
    for cell in FV
        i += 1
        tube = [v+k*offset for k in range(0, length=m+1) for v in cell]
        cellTube = [tube[k:k+d] for k in range(1, length=rangelimit)]
        if i==1 outcells = reshape(cellTube, d, m)
        else outcells = vcat(outcells, reshape(cellTube, d, m)) end
    end
    cellGroups = []
    for i in 1:size(outcells, 2)
        if pattern[i]>0
            cellGroups = vcat(cellGroups, outcells[:, i])
        end
    end
    outVertices = [vcat(v, [z]) for z in coords for v in V]
    cellGroups = convert(Array{Array{Int, 1}, 1}, cellGroups)
    outModel = outVertices, cellGroups
    hcat(outVertices...), cellGroups
end

extrudeSimplicial (generic function with 2 methods)

In [39]:
pattern = repeat([1,2,-3],outer=4)
@btime extrudeSimplicial(([[0,0] [1,0] [2,0] [0,1] [1,1] [2,1] [0,2] [1,2] [2,2]], [[1,2,4],[2,3,5],[3,5,6],[4,5,7],[5,7,8],[6,8,9]]), pattern)

  35.723 μs (560 allocations: 67.77 KiB)


([0 1 … 1 2; 0 0 … 2 2; 0 0 … 24 24], [[1, 2, 4, 10], [2, 4, 10, 11], [4, 10, 11, 13], [2, 3, 5, 11], [3, 5, 11, 12], [5, 11, 12, 14], [3, 5, 6, 12], [5, 6, 12, 14], [6, 12, 14, 15], [4, 5, 7, 13]  …  [96, 102, 104, 105], [94, 95, 97, 103], [95, 97, 103, 104], [97, 103, 104, 106], [95, 97, 98, 104], [97, 98, 104, 106], [98, 104, 106, 107], [96, 98, 99, 105], [98, 99, 105, 107], [99, 105, 107, 108]])

## simplexGrid(shape::Array)::LAR
Generate a simplicial complex decomposition of a cubical grid of ``d``-cuboids, where ``d`` is the length of `shape` array.

In [6]:
function simplexGrid(shape)
    model = [[]], [[1]]
    for item in shape
        model = extrudeSimplicial(model, fill(1, item))
    end
    V, CV = model
    V = convert(Array{Float64,2}, V)
    return V, CV
end

simplexGrid (generic function with 1 method)

In [8]:
@btime simplexGrid([1,1,1])

  7.637 μs (138 allocations: 9.47 KiB)


([0.0 1.0 … 0.0 1.0; 0.0 0.0 … 1.0 1.0; 0.0 0.0 … 1.0 1.0], [[1, 2, 3, 5], [2, 3, 5, 6], [3, 5, 6, 7], [2, 3, 4, 6], [3, 4, 6, 7], [4, 6, 7, 8]])

## quads2triangles(quads::Cells)::Cells
Convert an array of *quads* with type `::Lar.Cells` into an array of *triangles* with the same type.

In [25]:
function quads2triangles(quads::Lar.Cells)::Lar.Cells
	pairs = [[ Int[v1,v2,v3], Int[v3,v2,v4]] for (v1,v2,v3,v4) in quads ]
	return cat(pairs)
end

quads2triangles (generic function with 1 method)

In [35]:
V,FV = Lar.cuboidGrid([4,5])
triangles = quads2triangles(FV)

LoadError: UndefKeywordError: keyword argument dims not assigned

## simplexFacets(simplices::Vector{Int64})::Vector{Vector{Int64}}
Compute the `(d-1)`-skeleton (unoriented set of `facets`) of a simplicial `d`-complex.

In [15]:
function simplexFacets(simplices)
    out = Array{Int64,1}[]
	for simplex in simplices
		for v in simplex
			facet = setdiff(simplex,v)
			push!(out, facet)
		end
	end
	# remove duplicate facets
	return sort(collect(Set(out)))
end

simplexFacets (generic function with 1 method)

In [22]:
@btime simplexFacets([[1, 2, 3, 5],[2, 3, 5, 6],[3, 5, 6, 7],[2, 3, 4, 6],[3, 4, 6, 7],[4, 6, 7, 8]])

  8.703 μs (140 allocations: 14.29 KiB)


18-element Vector{Vector{Int64}}:
 [1, 2, 3]
 [1, 2, 5]
 [1, 3, 5]
 [2, 3, 4]
 [2, 3, 5]
 [2, 3, 6]
 [2, 4, 6]
 [2, 5, 6]
 [3, 4, 6]
 [3, 4, 7]
 [3, 5, 6]
 [3, 5, 7]
 [3, 6, 7]
 [4, 6, 7]
 [4, 6, 8]
 [4, 7, 8]
 [5, 6, 7]
 [6, 7, 8]

In [ ]:
prova